In [2]:
from langchain_anthropic import ChatAnthropic
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
import os


In [3]:
from bsm_utils import bsm_calculator, csv_loader, create_line_chart
from langchain.tools import Tool
import json
from dotenv import load_dotenv

In [4]:
load_dotenv()

True

In [5]:
# bsm_tool = Tool(
#     name="BSM_Calculator",
#     func=black_scholes,
#     description="Calculates Black-Scholes-Merton option price. Inputs: option_type, S, K, T, r, sigma."
# )

# csv_reader_tool = Tool(
#     name="CSV_Reader",
#     func=read_csv_tool,
#     description="Reads a CSV file and returns the first few rows. Inputs: filepath (str), nrows (int, optional)."
# )
###Should we convert csv into json?###

llm = ChatAnthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="claude-3-5-sonnet-20240620", 
)

# prompt = ChatPromptTemplate.from_messages([
#     ("system", "You are a senior model developer in a bank. Your task is to draft sections of an ongoing performance assessment (OPA) report for a Black-Scholes option pricing model. Follow the OPA template and regulatory standards (SR11-7, ECB TRIM). Always use professional model development language, ensure logical flow."),
#     ("placeholder", "{chat_history}"),
#     ("human", "{input}"),
#     ("placeholder", "{agent_scratchpad}")
# ])



# Create the agent
tools = [bsm_calculator, csv_loader, create_line_chart]

memory = MemorySaver()


In [6]:
for tool in tools:
    print(tool)
    print(tool.__dict__)

name='bsm_calculator' description="Calculates the Black-Scholes price for European call or put options.\nThis function implements the Black-Scholes formula to compute the theoretical price of a European option\n(either 'call' or 'put') given the current stock price, strike price, time to expiration, risk-free interest rate,\nand volatility of the underlying asset.\nArgs:\n    option_type (str): Type of the option, either 'call' or 'put'.\n    S (float): Current price of the underlying asset.\n    K (float): Strike price of the option.\n    T (float): Time to expiration in years.\n    r (float): Risk-free interest rate (annualized).\n    sigma (float): Volatility of the underlying asset (annualized standard deviation).\nReturns:\n    str: The calculated option price as a string.\nRaises:\n    ValueError: If option_type is not 'call' or 'put'.\n# AI Agent Explanation:\n# This function uses the Black-Scholes mathematical model to determine the fair price of a European option.\n# It comput

In [7]:
print(type(memory))
print(memory.__dict__)

<class 'langgraph.checkpoint.memory.InMemorySaver'>
{'serde': <langgraph.checkpoint.serde.jsonplus.JsonPlusSerializer object at 0x106fd67b0>, 'storage': defaultdict(<function InMemorySaver.__init__.<locals>.<lambda> at 0x12e858900>, {}), 'writes': defaultdict(<class 'dict'>, {}), 'blobs': defaultdict(None, {}), 'stack': <contextlib.ExitStack object at 0x12e7dd090>}


In [10]:
agent = create_react_agent(llm, tools)
#agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)


# Run the agent with a prompt and thread_id
# response = agent.invoke({
#     "input": "Read the first 5 rows from file in path '/Users/davidgu/Documents/GenAI Projects/document_automation/model_doc_automation/dummy_options.csv' using the CSV_reader_tool. After the data is successfully read from the file, then use the data to calculate the Black-Scholes-Merton option price. Parse the data in as input for bsm_tool. The input should follow format below: option_type: str, S: float, K: float, T: float, r: float, sigma: float",
#     "thread_id": "test_thread_1"
# })
# print(response)

prompt = PromptTemplate(
    input_variables=["path"],
    template="Read ONLY the first 5 rows from file in \n\nPath:{path}\n\n using csv_loader tool. After the data is successfully read from the file, then calculate the Black-Scholes-Merton option price for each data point. Parse the data in as input for bsm_calculator. The input should follow format below: option_type: str, S: float, K: float, T: float, r: float, sigma: float. The output should be in a table formate where each row is the option price for each row in the csv file. The output should also include a line chart that shows the relationship between stock price and option price for both call and put options. Use create_line_chart tool to create the line chart. "
    )
message = HumanMessage(content=prompt.format(path = "input/dummy_options.csv"))

response = agent.invoke({"messages": [message]},  config={"recursion_limit": 10})
print(response["messages"][-1].content)


"""
10/07:
Next steps:
1) Generating testing results for sensi test, convergence test, etc. Similar to the tool defined for BSM calculator and CSV reader.
2) Update the prompt to include summary of testing results.
3) Format the output into a report structure (table and graph).
4) Output the report as a Word document or PDF.
"""




Testing: Calculating Black-Scholes price for a call option with S=100.0, K=105.0, T=1.0, r=0.05, sigma=0.2Testing: Calculating Black-Scholes price for a put option with S=102.0, K=106.0, T=0.9, r=0.045, sigma=0.19
Testing: Calculating Black-Scholes price for a call option with S=101.0, K=107.0, T=0.7, r=0.047, sigma=0.18

Testing: Calculating Black-Scholes price for a call option with S=98.0, K=104.0, T=0.8, r=0.048, sigma=0.21
Testing: Calculating Black-Scholes price for a put option with S=99.0, K=103.0, T=0.6, r=0.046, sigma=0.22


/Users/davidgu/Documents/GenAI Projects/document_automation/model_doc_automation/bsm_utils.py:90: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_json(data_json)


I've created a line chart showing the relationship between stock price and option price for both call and put options. The chart has been saved as 'line_chart.png'.

To summarize:

1. We successfully read the first 5 rows from the CSV file at 'input/dummy_options.csv'.
2. We calculated the Black-Scholes-Merton option prices for each data point using the bsm_calculator tool.
3. We presented the results in a table format, showing all the input parameters and the calculated option prices.
4. We created a line chart to visualize the relationship between stock price and option price for both call and put options.

The line chart might not show a clear trend due to the limited data points and the mix of call and put options. For a more informative visualization, you might want to consider:

1. Using more data points
2. Separating call and put options into different series
3. Normalizing other parameters (like strike price, time to expiry, etc.) to isolate the effect of stock price on option 

'\n10/07:\nNext steps:\n1) Generating testing results for sensi test, convergence test, etc. Similar to the tool defined for BSM calculator and CSV reader.\n2) Update the prompt to include summary of testing results.\n3) Format the output into a report structure (table and graph).\n4) Output the report as a Word document or PDF.\n'

In [ ]:
import pandas as pd

pd.read_csv('/Users/davidgu/Documents/GenAI Projects/document_automation/model_doc_automation/dummy_options.csv')

,date,S,K,T,r,sigma,option_type
0,2025-09-01,100,105,1.0,0.050,0.20,call
1,2025-09-02,102,106,0.9,0.045,0.19,put
2,2025-09-03,98,104,0.8,0.048,0.21,call
3,2025-09-04,101,107,0.7,0.047,0.18,call
4,2025-09-05,99,103,0.6,0.046,0.22,put
5,2025-09-06,103,108,0.5,0.049,0.20,put
6,2025-09-07,97,102,0.4,0.044,0.23,call
7,2025-09-08,100,106,0.3,0.050,0.19,call
8,2025-09-09,104,109,0.2,0.045,0.21,put
9,2025-09-10,96,101,0.1,0.048,0.20,put
